### 11/01,これからも再現性が気になることはあると思うので、再現性チェックに関するコードを作成したいと思った
* 何をもって再現性が取れている、とするんだ？ --

In [3]:
import pandas as pd
import numpy as np
import torch

from glob import glob
from tqdm import tqdm

In [4]:
path1 = "/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_001/*.pth"
path2 = "/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_002/*.pth"

In [9]:
model_paths1 = glob(path1)
model_paths2 = glob(path2)
model_paths1.sort()
model_paths2.sort()

In [10]:
model_paths1

['/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_001/checkpoint-fold0.pth',
 '/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_001/checkpoint-fold1.pth']

In [11]:
model_paths2

['/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_002/checkpoint-fold0.pth',
 '/mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_002/checkpoint-fold1.pth']

In [13]:
fold = 0
m1 = torch.load(model_paths1[fold])["model_state_dict"]
m2 = torch.load(model_paths2[fold])["model_state_dict"]

In [46]:
torch.nn.init.zeros_(m2["l1.embeddings.LayerNorm.bias"])

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

In [74]:
from typing import OrderedDict
from colorama import Fore
g_ = Fore.GREEN
y_ = Fore.YELLOW
r_ = Fore.RED
sr_ = Fore.RESET

def torch_parameter_compare(m1_path:str, m2_path:str) -> list:
    """2つのpthのパスを受け取り、state_dictの各キーごとにtensorの値が一致するかどうかを確認する関数"""
    print(f"Compare with")
    print(f"    --> {m1_path}")
    print(f"    --> {m2_path}")

    # pthのロード --
    m1 = torch.load(m1_path)["model_state_dict"]
    m2 = torch.load(m2_path)["model_state_dict"]

    # そもそもkeyが完全一致しなければ計算しない, modelのstructureを確認してほしい --
    assert (len(set(m1.keys()) - set(m2.keys())) == 0), "... m1.keys() vs m2.keys() dosen't match ..."

    # 各キーごとにtensorを比較する --
    unmatched_keys = []
    for key in m1.keys():
        key_matches = torch.eq(m1[key], m2[key]).all().item()
        if not key_matches:
            print(f"{g_}{key} --> {r_}{key_matches}{sr_}")
            unmatched_keys.append(key)

    if len(unmatched_keys) == 0:
        print(f"{g_}        --> All keys matched ... {sr_}")
    else:
        print(f"{y_}        --> Some keys unmatched : {unmatched_keys} {sr_}")
    
    return unmatched_keys

In [75]:
for fold in range(0, 2):
    _ = torch_parameter_compare(
        model_paths1[fold], 
        model_paths2[fold]
        )

Compare with
    --> /mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_001/checkpoint-fold0.pth
    --> /mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_002/checkpoint-fold0.pth
        --> All keys matched ... 
Compare with
    --> /mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_001/checkpoint-fold1.pth
    --> /mnt/sdb/NISHIKA_DATA/hate-speech-detection/output/dropout-seed_002/checkpoint-fold1.pth
        --> All keys matched ... 
